In [1]:
import numpy as np
import re
import os
import sys

In [2]:
filename = 'test_AutoGeneratedCaption.vtt'

In [3]:
def read_message(i, data):
    my_message={}
    my_message['confidence']= data[i].split(':')[1].strip()
    i+=2
    my_message['marker'] = data[i].strip()
    i+=1
    my_message['start'] = data[i].split(' ')[0].strip()
    my_message['stop'] = data[i].split(' ')[-1].strip()
    my_message['content'] = []
    while data[i] != '\n':
        i+=1 
        if data[i] != '\n':
            my_message['content'].append(data[i].strip())
    return my_message

In [4]:
def create_dictionary(filename):
    with open(filename) as file:
        data = [line for line in file]
        data_dict = {}
        data_dict['messages'] = []
        for i, line in enumerate(data):
            if line.startswith( 'NOTE duration' ):
                data_dict['duration'] = line.split(':"')[1].strip()
            if line.startswith( 'NOTE language' ):
                data_dict['language'] = line.split(':')[1].strip()
            if line.startswith('NOTE Confidence'):
                data_dict['messages'].append(read_message(i, data))
    return data_dict

def format_text(filename):
    data_dict = create_dictionary(filename)
    part_messages = []
    for item in data_dict['messages']:   
        part_messages.append(' '.join(item['content']))
    
    full_messages = []
    start = [item['start'] for item in data_dict['messages']]
    stop =  [item['stop'] for item in data_dict['messages']]
    
    x=np.array([start, stop, part_messages])

    i=0
    while i < len(start)-2:
        if x[0,i+1] != x[1,i]:
            full_messages.append(x[2,i])
            i+=1
        else:
            sentence = ''
            while x[0,i+1] == x[1,i]:
                sentence = sentence + x[2,i] + ' '
                i+=1
            sentence = sentence + x[2,i]
            i+=1
            full_messages.append(sentence)
        
        
    if x[0,-1] == x[1,-2]:
        end = x[2,-2] + ' ' + x[2,-1]
        full_messages.append(end)
    else:
        full_messages.append(x[2,-2])
        full_messages.append(x[2,-1])
        
    return full_messages

def reformat_vtt(filename):
    text = format_text(filename)
    newfile = open(os.path.join(filename.replace(".vtt", ".txt")),'w')
    for line in text:
        newfile.write("%s" % line)
        newfile.write("\n \n")

In [5]:
reformat_vtt(filename)

In [6]:
re_uuid = re.compile(r'[0-9a-f]{8}(?:-[0-9a-f]{4}){3}-[0-9a-f]{12}', re.I)
df['identifier'] = [ re.findall(re_uuid, line) for line in data if re.findall(re_uuid, line) ]


NameError: name 'data' is not defined